In [6]:
from bs4 import BeautifulSoup
import requests
import re
import sys
import os
sys.path.append(os.path.abspath("../scripts"))
from currentSeason import find_current_season





playerURL = "http://onlinecollegebasketball.org/player/190887"

page = requests.get(playerURL)
soup = BeautifulSoup(page.text, "html.parser")


#infoList = soup.find("table").find_all("tr")[4:11]

fullinfoList = soup.find("table").find_all("tr")

for i in range(len(fullinfoList)):
    if "Age" in fullinfoList[i].text:
        start_index = i
    elif "Perimeter Defense" in fullinfoList[i].text:
        end_index = i + 1

infoList = fullinfoList[start_index:end_index]



def split_number_and_letter(s):
    s = s.replace('↑', '')
    # Split the string into parts that contain numbers, fractions, letters, single quotes, and double quotes
    return re.findall(r'\d+|[¼½]|[^\d¼½\'"]+|[\'"]', s)



def extract_length(array):
    length = ""
    lengthArr = array[1:-4]
    for i in lengthArr:
        length += i
    
    return length



#For Height and Wingspan
def convert_to_inches(height):
    parts = height.split("'")
    if len(parts) == 2:
        feet = int(parts[0])
        inches_str = parts[1].rstrip('"').strip()  # Removing leading/trailing whitespace
        if inches_str:  # Check if inches part is not empty
            if '½' in inches_str:
                inches_str = inches_str.replace('½', '.5')
            elif '¼' in inches_str:
                inches_str = inches_str.replace('¼', '.25')
            inches = float(inches_str)
        else:
            inches = 0
    elif len(parts) == 1:
        feet = int(parts[0])
        inches = 0
    else:
        return None
    return (feet * 12) + inches
    

#For Vertical
def Vert_convert_to_inches(height):
    inches_str = height.rstrip('"')  # Removing the inch symbol
    if '½' in inches_str:
        inches_str = inches_str.replace('½', '.5')
    elif '¼' in inches_str:
        inches_str = inches_str.replace('¼', '.25')
    return float(inches_str)





player_info = {}

for i in range(len(infoList)):
    text = infoList[i].text.strip().replace(" ", "")
    
    if i == 1:
        find_Outside = text.find("O")
        
        separated = split_number_and_letter(text[find_Outside:])
    else:
        separated = split_number_and_letter(text)

    if separated:
        


        if 'Height' in separated[0]:
            separated = separated[-4:]
        if 'Wingspan' in separated[0]:
            separated = separated[-4:]
        if 'Vertical' in separated[0]:
            separated = separated[-4:]
        
        # Handle special cases
        if 'Weight' in separated[0]:
            player_info['Weight'] = float(f"{separated[1]}")
            
            start_idx = 2
            separated[start_idx] = separated[start_idx].replace("lbs.","")

        else:
            start_idx = 0

        # Add remaining key-value pairs to the dictionary
        for j in range(start_idx, len(separated), 2):
            key = separated[j].replace(":", "")
            if j + 1 < len(separated):
                
                value = separated[j + 1]
                player_info[key] = int(value)






player_info["Height_inches"] = convert_to_inches(extract_length(split_number_and_letter(infoList[2].text.strip().replace(" ", ""))))
player_info["Wingspan_inches"] = convert_to_inches(extract_length(split_number_and_letter(infoList[4].text.strip().replace(" ", ""))))

player_info["Vertical_float"] = Vert_convert_to_inches(extract_length(split_number_and_letter(infoList[5].text.strip().replace(" ", ""))))








print(player_info)




{'Age': 22, 'InsideShot': 7, 'BasketballIQ': 14, 'OutsideShot': 12, 'Passing': 17, 'ShootingRange': 17, 'BallHandling': 15, 'Weight': 200.0, 'Finishing': 8, 'Driving': 19, 'Rebounding': 6, 'Strength': 11, 'InteriorDefense': 11, 'Speed': 18, 'PerimeterDefense': 10, 'Stamina': 10, 'Height_inches': 73.5, 'Wingspan_inches': 76.25, 'Vertical_float': 26.5}


In [7]:
def transform_player_data(input_data):
    # Define the mapping from input keys to required keys
    key_mapping = {
        'Age': None,  # Not used in the output
        "Height_inches" : "Height_inches",
        'Weight': 'Weight',
        "Wingspan_inches": "Wingspan_inches",
        "Vertical_float": "Vertical_float",
        'InsideShot': 'IS',
        'BasketballIQ': 'IQ',
        'OutsideShot': 'OS',
        'Passing': 'Pass',
        'ShootingRange': 'Rng',
        'BallHandling': 'Hnd',
        'Finishing': 'Fin',
        'Driving': 'Drv',
        "Rebounding": 'Reb',
        'Strength': 'Str',
        'InteriorDefense': 'IDef',
        'Speed': 'Spd',
        'PerimeterDefense': 'PDef',
        'Stamina': 'Sta',
       
    }
    
    # Initialize the output dictionary
    output_data = {}
    

    
    # Apply the mapping to transform the input dictionary
    for key, new_key in key_mapping.items():
        if new_key is not None and key in input_data:
            output_data[new_key] = input_data[key]
    


    output_data["Player_ID"] = int(playerURL.split("/")[-1])

    season = find_current_season()
    
    output_data["Season_ID"] = season

    #Get Player Name/Team ID
    soup = BeautifulSoup(page.text, "html.parser")
    #print(soup)

    pageList = soup.find("th").find_all("a")[1:]

    player_name = pageList[0].find("img")["title"].replace("Bookmark ","")

    team_code = int(pageList[-1].get("href").split("/")[-1])

    output_data["Name"] = player_name
    output_data["Team_ID"] = team_code

    return output_data


transform_player_data(player_info)

{'Height_inches': 73.5,
 'Weight': 200.0,
 'Wingspan_inches': 76.25,
 'Vertical_float': 26.5,
 'IS': 7,
 'IQ': 14,
 'OS': 12,
 'Pass': 17,
 'Rng': 17,
 'Hnd': 15,
 'Fin': 8,
 'Drv': 19,
 'Reb': 6,
 'Str': 11,
 'IDef': 11,
 'Spd': 18,
 'PDef': 10,
 'Sta': 10,
 'Player_ID': 190887,
 'Season_ID': 2043,
 'Name': 'Geraldo Munoz',
 'Team_ID': 533}

In [8]:
soup = BeautifulSoup(page.text, "html.parser")
#print(soup)

pageList = soup.find("th").find_all("a")[1:]

player_name = pageList[0].find("img")["title"].replace("Bookmark ","")

team_code = int(pageList[-1].get("href").split("/")[-1])

transform_player_data(player_info)["Name"] = player_name
transform_player_data(player_info)["Team_ID"] = team_code
transform_player_data


<function __main__.transform_player_data(input_data)>